In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sqlalchemy import create_engine


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from xgboost import XGBClassifier

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE


pd.options.display.max_rows = 200
pd.options.display.max_columns = 200

UNKNOWN = 'unknown'
RANDOM_STATE = 2021

In [2]:
path = 'sqlite:///../data/processed/'

# Import protest and countries data (already merged)
engine = create_engine(path+'merged.db')
with engine.begin() as connection:
    df = pd.read_sql('SELECT * FROM merged', con=connection)

In [3]:
# Relace 'placeholder' NaN values, as defined by data dictionary (see raw data directory)
df.replace(-999, np.nan, inplace=True)
df.replace(-999.0, np.nan, inplace=True)
df.replace('-999', np.nan, inplace=True)
df.replace('-999.0', np.nan, inplace=True)
print(df.iloc[:, :100].info())
print(df.iloc[:, 100:].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15140 entries, 0 to 15139
Data columns (total 100 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   id                                  15140 non-null  int64  
 1   country                             15140 non-null  object 
 2   location                            15140 non-null  object 
 3   region                              15140 non-null  object 
 4   protestnumber                       15140 non-null  int64  
 5   protesterviolence                   15140 non-null  int64  
 6   protesteridentity                   15140 non-null  object 
 7   startyear                           15140 non-null  int64  
 8   startmonth                          15140 non-null  int64  
 9   startday                            15140 non-null  int64  
 10  endyear                             15140 non-null  int64  
 11  endmonth                            1514

In [4]:
df.head(3)

,id,country,location,region,protestnumber,protesterviolence,protesteridentity,startyear,startmonth,startday,endyear,endmonth,endday,start_datetime,end_datetime,duration_days,participants,participants_category_original,participants_category_manufactured,sources,notes,demand_labor-wage-dispute,demand_land-farm-issue,demand_police-brutality,demand_political-behavior/process,demand_price-increases/tax-policy,demand_removal-of-politician,demand_social-restrictions,response_accomodation,response_arrests,response_beatings,response_crowd-dispersal,response_ignore,response_killings,response_shootings,system,yrsoffc,finittrm,yrcurnt,termlimit,reelect,multpl,military,defmin,percent1,percentl,prtyin,execme,execrlc,execnat,execrurl,execreg,execrel,execage,allhouse,nonchief,totalseats,gov1me,gov1seat,gov1vote,gov1rlc,gov1nat,gov1rurl,gov1reg,gov1rel,gov1age,gov2me,gov2seat,gov2vote,gov2rlc,gov2nat,gov2rurl,gov2reg,gov2rel,gov2age,gov3me,gov3seat,gov3vote,gov3rlc,gov3nat,gov3rurl,gov3reg,gov3rel,gov3age,govoth,govothst,govothvt,opp1me,opp1seat,opp1vote,opp1rlc,opp1nat,opp1rurl,opp1reg,opp1rel,opp1age,opp2me,opp2seat,opp2vote,opp3me,opp3seat,opp3vote,oppoth,oppothst,oppothvt,ulprty,numul,ulvote,oppmajh,oppmajs,dateleg,dateexec,legelec,exelec,liec,eiec,mdmh,mdms,ssh,pluralty,pr,housesys,sensys,thresh,dhondt,cl,select,gq,gqi,fraud,auton,muni,state,author,stconst,gwno,numgov,numvote,numopp,oppvote,maj,partyage,herfgov,herfopp,herftot,frac,oppfrac,govfrac,tensys_strict,tensys,checks_lax,checks,stabs_strict,stabs,stabns_strict,stabns,tenlong_strict,tenlong,tenshort_strict,tenshort,polariz
0,201990001,Canada,national,North America,1,0,unknown,1990,1,15,1990,1,15,1990-01-15 00:00:00.000000,1990-01-15 00:00:00.000000,0,1000,unknown,1000-1999,1. great canadian train journeys into history;...,canada s railway passenger system was finally ...,1,0,0,1,0,0,0,0,0,0,0,1,0,0,Parliamentary,6.0,1.0,3.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,6.0,PCP,Right,0.0,0.0,0.0,0.0,136.0,0.0,NaN,295.0,PCP,169.0,43.009998,Right,0.0,0.0,0.0,0.0,136.0,None,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,None,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,LPC,83.0,31.91,Left,0.0,0.0,0.0,0.0,NaN,NDP,43.0,20.379999,independent,1.0,0.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,7.0,7.0,1.0,888.0,0.28,1.0,0.0,Plurality,NaN,NaN,NaN,NaN,None,3.0,0.0,0.0,0.0,Legislature and executive locally elected,Legislature and executive locally elected,1.0,1.0,20.0,169.0,43.009998,127.0,52.459999,0.570945918560028,136.0,1.0,0.541819095611572,None,0.576225340366364,0.461817264556885,0.0,60.0,60.0,4.0,4.0,0.0,0.0,0.0,0.0,6.0,6.0,6.0,6.0,0.0
1,201990002,Canada,"Montreal, Quebec",North America,2,0,unknown,1990,6,25,1990,6,25,1990-06-25 00:00:00.000000,1990-06-25 00:00:00.000000,0,1000,unknown,1000-1999,1. autonomy s cry revived in quebec the new yo...,protestors were only identified as young peopl...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,Parliamentary,6.0,1.0,3.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,6.0,PCP,Right,0.0,0.0,0.0,0.0,136.0,0.0,NaN,295.0,PCP,169.0,43.009998,Right,0.0,0.0,0.0,0.0,136.0,None,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,None,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,LPC,83.0,31.91,Left,0.0,0.0,0.0,0.0,NaN,NDP,43.0,20.379999,independent,1.0,0.17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,7.0,7.0,1.0,888.0,0.28,1.0,0.0,Plurality,NaN,NaN,NaN,NaN,None,3.0,0.0,0.0,0.0,Legislature and executive locally elected,Legislature and executive locally elected,1.0,1.0,20.0,169.0,43.009998,127.0,52.459999,0.570945918560028,136.0,1.0,0.541819095611572,None,0.576225340366364,0.461817264556885,0.0,60.0,60.0,4.0,4.0,0.0,0.0,0.0,0.0,6.0,6.0,6.0,6.0,0.0
2,201990003,Canada,"Montreal, Quebec",North America,3,0,separatist parti quebecois,1990,7,1,1990,7,1,1990-07-01 00:00:00.000000,1990-07-01 00:00:00.000000,0,500,unknown,100-999,1. quebec protest after queen calls for unity ...,"the queen, after calling on canadians to remai...",0,0,0,1,0,0,0,0,0,0,0,1,0,0,Parliamentary,6.0,1.0,3.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,6.0,PCP,Right,0.0,0.0,0.0,0.0,136.0,

In [5]:
# Set semi-arbitrary threshold for the maximum number of missing values to justify keeping
MAX_MISSING_VALUES = 1000

# Copy main df to retain a full dataset
df_cut = df.copy()

# Determine the number of missing values in each column
na_counts_by_col = {}
for col in df_cut.columns:
    na_ct = df_cut[col].isna().sum()
    na_counts_by_col[col] = na_ct # Keep record in dictionary in case you want to investigate
    
    if na_ct > MAX_MISSING_VALUES:
        df_cut.drop(col, axis=1, inplace=True)
        
df_cut.dropna(inplace=True)
df_cut.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11505 entries, 0 to 15139
Data columns (total 95 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   id                                  11505 non-null  int64  
 1   country                             11505 non-null  object 
 2   location                            11505 non-null  object 
 3   region                              11505 non-null  object 
 4   protestnumber                       11505 non-null  int64  
 5   protesterviolence                   11505 non-null  int64  
 6   protesteridentity                   11505 non-null  object 
 7   startyear                           11505 non-null  int64  
 8   startmonth                          11505 non-null  int64  
 9   startday                            11505 non-null  int64  
 10  endyear                             11505 non-null  int64  
 11  endmonth                            11505

In [6]:
df_cut['numgov'] = df_cut['numgov'].astype('float').astype('int')
df_cut['numopp'] = df_cut['numopp'].astype('float').astype('int')

df_cut['tensys'] = df_cut['tensys'].astype('float').astype('int')
df_cut['tensys_strict'] = df_cut['tensys_strict'].astype('float').astype('int')

df_cut['stabs'] = df_cut['stabs'].astype('float').astype('int')
df_cut['stabs_strict'] = df_cut['stabs_strict'].astype('float').astype('int')
df_cut['checks'] = df_cut['checks'].astype('float').astype('int')
df_cut['checks_lax'] = df_cut['checks_lax'].astype('float').astype('int')
df_cut['stabns'] = df_cut['stabns'].astype('float').astype('int')
df_cut['stabns_strict'] = df_cut['stabns_strict'].astype('float').astype('int')
df_cut['tenlong'] = df_cut['tenlong'].astype('float').astype('int')
df_cut['tenlong_strict'] = df_cut['tenlong_strict'].astype('float').astype('int')
df_cut['tenshort'] = df_cut['tenshort'].astype('float').astype('int')
df_cut['tenshort_strict'] = df_cut['tenshort_strict'].astype('float').astype('int')
df_cut['maj'] = df_cut['maj'].astype('float')
df_cut['frac'] = df_cut['frac'].astype('float')
df_cut['govfrac'] = df_cut['govfrac'].astype('float')
df_cut['herfgov'] = df_cut['herfgov'].astype('float')


df_cut.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11505 entries, 0 to 15139
Data columns (total 95 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   id                                  11505 non-null  int64  
 1   country                             11505 non-null  object 
 2   location                            11505 non-null  object 
 3   region                              11505 non-null  object 
 4   protestnumber                       11505 non-null  int64  
 5   protesterviolence                   11505 non-null  int64  
 6   protesteridentity                   11505 non-null  object 
 7   startyear                           11505 non-null  int64  
 8   startmonth                          11505 non-null  int64  
 9   startday                            11505 non-null  int64  
 10  endyear                             11505 non-null  int64  
 11  endmonth                            11505

# Run it through a model!

In [9]:
response_cols = ['response_accomodation', 'response_arrests', 'response_beatings', 'response_crowd-dispersal', 'response_ignore', 'response_killings', 'response_shootings']

model_inputs = df_cut.drop(response_cols+['id', 'location', 'protesteridentity', 'start_datetime', 'end_datetime', 'sources', 'notes', 'endday', 'endmonth', 'endyear', 'participants_category_original', 'country', 'execme', 'gov1me'], axis=1)
model_inputs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11505 entries, 0 to 15139
Data columns (total 74 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   region                              11505 non-null  object 
 1   protestnumber                       11505 non-null  int64  
 2   protesterviolence                   11505 non-null  int64  
 3   startyear                           11505 non-null  int64  
 4   startmonth                          11505 non-null  int64  
 5   startday                            11505 non-null  int64  
 6   duration_days                       11505 non-null  int64  
 7   participants                        11505 non-null  int64  
 8   participants_category_manufactured  11505 non-null  object 
 9   demand_labor-wage-dispute           11505 non-null  int64  
 10  demand_land-farm-issue              11505 non-null  int64  
 11  demand_police-brutality             11505

In [10]:
violent_cols = ['response_beatings', 'response_killings', 'response_shootings']
violent_response = df_cut[violent_cols].any(axis=1).astype('int')
accommodation = df_cut['response_accomodation']

y = accommodation ###### ***IMPORTANT LINE - CHOOSE WHICH TARGET TO USE*** #####
x_train, x_test, y_train, y_test = train_test_split(model_inputs, y, random_state=RANDOM_STATE)

### Define models and parameter grids

In [11]:
# Set parameter grid to search across
param_grid_logreg = {'model__C': np.logspace(-1, 5, 20)}


param_grid_dt = {
    'model__max_depth': [3, 5, 7], 
    'model__criterion': ['gini', 'entropy'],
    'model__min_samples_split': [5, 10],
    'model__min_samples_leaf': [5, 10]} 

param_grid_rf = {
    'model__n_estimators': [25, 75],#, 150],
    'model__criterion': ['gini', 'entropy'],
    'model__max_depth': [3, 7],# [3, 6, 10],
    'model__min_samples_split': [5, 10],
    'model__min_samples_leaf': [3, 6]}


param_grid_knn = {
    'model__leaf_size': [25, 50, 75],
    'model__n_neighbors': [3, 5, 7, 9]}#,
    #'model__weights': ['uniform', 'distance'],
    #'model__metric': ['euclindean', 'manhattan', 'minkowski']} 
        
        
param_grid_xgb = {
    'model__learning_rate': [0.1, 0.2],
    'model__max_depth': [3, 7],
    #'model__min_child_weight': [1, 2],
    #'model__subsample': [0.5, 0.7],
    'model__n_estimators': [100, 150]}#,
    #'model__tree_method': ['exact', 'approx', 'hist']}


np.random.seed(RANDOM_STATE)
model_logreg = LogisticRegression(max_iter=5000)
model_dt = DecisionTreeClassifier()
model_rf = RandomForestClassifier()
model_knn = KNeighborsClassifier()
model_xgb = XGBClassifier(eval_metric='logloss', use_label_encoder=False)

grids = [param_grid_logreg, param_grid_dt, param_grid_rf, param_grid_knn, param_grid_xgb]
models = [model_logreg, model_dt, model_rf, model_knn, model_xgb]

In [12]:
def create_pipeline_and_run(model, param_grid, metric='accuracy'):
    np.random.seed(RANDOM_STATE)
    ohe = OneHotEncoder(handle_unknown='error')
    scaler = StandardScaler()
    smote = SMOTE()

    selector_object = make_column_selector(dtype_include='object')
    selector_numeric = make_column_selector(dtype_include='number')
    transformer = make_column_transformer((ohe, selector_object),
                                         (scaler, selector_numeric))


    pipe = Pipeline([('transformer', transformer),
                     ('smote', smote), 
                     ('model', model)])

    # Instantiate and fit grid search object
    grid = GridSearchCV(pipe, param_grid, return_train_score=True, scoring=metric, cv=3)
    grid.fit(x_train, y_train)
    print(f'{model} {metric}: {grid.best_estimator_.score(x_test, y_test)}')
    return grid.best_estimator_

### Select your model and grid from above list to run one selected model

In [13]:
create_pipeline_and_run(model_logreg, param_grid_logreg);

LogisticRegression(max_iter=5000) accuracy: 0.6767466110531803


### Run below cell to iterate over *all* above-defined models

In [14]:
pipes = []
for grid, model in zip(grids, models):
    pipe = create_pipeline_and_run(model, grid)
    pipes.append(pipe)

LogisticRegression(max_iter=5000) accuracy: 0.6767466110531803
DecisionTreeClassifier() accuracy: 0.8849496002780675
RandomForestClassifier() accuracy: 0.8762599930483143
KNeighborsClassifier() accuracy: 0.7796315606534585
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              eval_metric='logloss', gamma=None, gpu_id=None,
              importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              use_label_encoder=False, validate_parameters=None,
              verbosity=None) accuracy: 0.9058046576294752


### Dummy classifier results

In [16]:
from sklearn.dummy import DummyClassifier

for strategy in ["stratified", "most_frequent", "uniform"]:
    dummy_clf = DummyClassifier(strategy=strategy, random_state=RANDOM_STATE)
    dummy_clf.fit(x_train, y_train)
    print(strategy, '-', dummy_clf.score(x_test, y_test))

stratified - 0.8196037539103233
most_frequent - 0.899895724713243
uniform - 0.49739311783107404
